In [1]:
%reload_ext autoreload
%autoreload 2

In [4]:
from image_pipeline.google_api_funcs import functions
import inspect 
from toolz import get_in, concat
from itertools import starmap
from functools import partial

import pprint

from typing import get_type_hints, Any,AnyStr, Union
from collections.abc import Sequence, Iterable, Mapping 

import pandas as pd

In [5]:
FUNCTIONS = { func.__name__ : func for func in functions.FUNCTIONS}
#{name: inspect.formatargspec(*inspect.getfullargspec(f)) for name, f in FUNCTIONS.items()}
{name: str(inspect.signature(f)) for name, f in FUNCTIONS.items()}

{'ident_landmarks': '(image_url: str) -> Any',
 'ident_crop_hints': '(*, image_url: str, aspect_ratios: Union[float, NoneType] = 1.0) -> image_pipeline.google_api_funcs.functions.CropHintAnnotation',
 'ident_dominant_colors': '(*, image_url: str, limit: Union[int, NoneType]) -> image_pipeline.google_api_funcs.functions.DominantColorAnnotation',
 'func_w_no_annotations': '(a)',
 'func_w_no_params': '()',
 'func_w_optional_annotations': '(a: Union[int, NoneType], b: Union[image_pipeline.google_api_funcs.functions.CropHintAnnotation, NoneType]) -> Union[image_pipeline.google_api_funcs.functions.CropHintAnnotation, NoneType]',
 'func_w_lists': '(a: List[int], b: List[int]) -> List[image_pipeline.google_api_funcs.functions.CropHintAnnotation]',
 'func_w_optional_elements': '(a: List[Union[int, NoneType]], b: List[Union[image_pipeline.google_api_funcs.functions.CropHintAnnotation, NoneType]]) -> List[Union[image_pipeline.google_api_funcs.functions.CropHintAnnotation, NoneType]]',
 'func_w_op

In [8]:
def resolve_base_types(atype):
    if atype == inspect._empty:
        return Any
    return atype

def get_function_types(func: callable):
    
    func_name = func.__name__
    signiture =  inspect.signature(func)
    return_type = resolve_base_types(signiture.return_annotation)
    parameters = { name: resolve_base_types(param_type.annotation)
                    for name, param_type in signiture.parameters.items()}

    
    # Output return record
    yield func_name, 'out', func_name, func_name, return_type
    
    # Input parameter records
    for param_sname, parameter in signiture.parameters.items():
        param_type = resolve_base_types(parameter.annotation)
        param_name = f"{func_name}.{param_sname}"
        yield func_name, 'in', param_sname, param_name, param_type
reader = map(get_function_types,  FUNCTIONS.values())
reader = concat(reader)
df = pd.DataFrame(reader, columns=['function','input_output','parameter_sname','parameter_name', 'parameter_type']).set_index('parameter_name')

def is_type_subclass(val, klass):
    try:
        return issubclass(val, klass)
    except Exception as e:
        return False
    
def args_have_none(val):
    if val is None:
        return None
    return sum(1 for x in val if getattr(x, '__name__', None) == 'NoneType')
    
df['origin'] = df.parameter_type.map( lambda x: getattr(x, '__origin__', None))
df['args'] = df.parameter_type.map( lambda x: getattr(x,'__args__', None))
df['type_name'] = df.parameter_type.map(lambda x: x.__dict__.get( '_name'))
df['is_iterable' ] = df.origin.map( partial(is_type_subclass, klass=Iterable))
df['is_sequence' ] = df.origin.map( partial(is_type_subclass, klass=Sequence))
df['is_union' ] = df.origin.map( lambda x: x == Union)
df['none_args'] = df.args.map(args_have_none)

FUNC_INFO = df
#FUNC_INFO
df.iloc[:,  3:]

FUNC_INFO.to_csv('metadata_from_typing.csv')

In [51]:
x = FUNC_INFO.loc['func_w_optional_lists']

y = x.args[1]
inspect.getmembers(y)

y.__name__

'NoneType'

In [96]:
def get_referenced_types(function_types):
    return function_types

get_referenced_types(function_types)

reader = ((name, 'in', param_name, param_type ) for name, func_def in function_types.items() for param_name, param_type in func_def['in'].items())
reader = ((name, 'out','return', func_def['out']) for name, func_def in function_types.items())
list(reader)

[('ident_landmarks', 'out', 'return', typing.Any),
 ('ident_crop_hints',
  'out',
  'return',
  image_pipeline.google_api_funcs.functions.CropHintAnnotation),
 ('ident_dominant_colors',
  'out',
  'return',
  image_pipeline.google_api_funcs.functions.DominantColorAnnotation),
 ('func_w_no_annotations', 'out', 'return', typing.Any),
 ('func_w_no_params', 'out', 'return', typing.Any)]

In [60]:
def add_if_exists( source, attr_name, adict, new_name):
    value = getattr(source, attr_name, None)
    if value is not None:
        adict[new_name] = value
def resolve_type_class(type_obj):
    
    output = {}
    add_if_exists(type_obj, '__name__', output, 'name')
    add_if_exists(type_obj, '__args__', output, 'args')
    add_if_exists(type_obj, '__origin__', output, 'origin')
    
    dataclass_fields = getattr(type_obj, '__dataclass_fields__', None)
    if dataclass_fields is not None:
        output['dataclass_fields'] = {field.name : resolve_type_class(field.type) for field in type_obj.__dataclass_fields__.values()}
    output['type_obj'] =  type_obj
    return output


pprint.pprint(y1)

#x = get_in(['returns' ,'dataclass_fields', 'colors','type_obj'], y1)

{'inputs': {'image_url': <class 'str'>, 'limit': typing.Union[int, NoneType]},
 'name': 'ident_dominant_colors',
 'output': <class 'image_pipeline.google_api_funcs.functions.DominantColorAnnotation'>}


In [55]:
x.__origin__

list

In [35]:
print(x.__args__)
print(x._name)
x.__origin__

(<class 'image_pipeline.google_api_funcs.functions.DominantColorEstimate'>,)
List


list

In [1]:
from image_pipeline import schemas
from image_pipeline import schemas_pydantic
from image_pipeline import schemas2
from pydantic.schema import schema

In [11]:
print(schemas_pydantic.AnnotatedImage.schema_json(indent=2))

{
  "title": "AnnotatedImage",
  "type": "object",
  "properties": {
    "image_url": {
      "title": "Image_Url",
      "minLength": 1,
      "maxLength": 65536,
      "type": "string",
      "format": "uri"
    },
    "comment": {
      "title": "Comment",
      "type": "string"
    },
    "dominant_colors": {
      "title": "Dominant_Colors",
      "type": "array",
      "items": {
        "$ref": "#/definitions/DominantColor"
      }
    },
    "bounding_boxes": {
      "title": "Bounding_Boxes",
      "type": "array",
      "items": {
        "$ref": "#/definitions/BoundingBox"
      }
    },
    "landmarks": {
      "title": "Landmarks",
      "type": "array",
      "items": {
        "$ref": "#/definitions/Landmark"
      }
    }
  },
  "required": [
    "image_url"
  ],
  "definitions": {
    "DominantColor": {
      "title": "DominantColor",
      "type": "object",
      "properties": {
        "fractionOfPixels": {
          "title": "Fractionofpixels",
          "type": "nu

In [38]:
schemas2.AnnotatedImage

mappingproxy({'__module__': 'image_pipeline.schemas2',
              '__doc__': 'AnnotatedImage(image_url: str, dominant_colors: Union[List[image_pipeline.schemas2.DominantColor], NoneType], bounding_boxes: Union[List[image_pipeline.schemas2.BoundingBox], NoneType], landmarks: Union[List[image_pipeline.schemas2.Landmark], NoneType])',
              '__post_init__': <function pydantic.dataclasses._pydantic_post_init(self: 'DataclassType') -> None>,
              '__dataclass_params__': _DataclassParams(init=True,repr=True,eq=True,order=False,unsafe_hash=False,frozen=True),
              '__dataclass_fields__': {'image_url': Field(name='image_url',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x10cf89128>,default_factory=<dataclasses._MISSING_TYPE object at 0x10cf89128>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD),
               'dominant_colors': Field(name='dominant_colors',type=typing.Union[typing.List[image_pipeline.schem